In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import glob

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
df=pd.read_csv(r"C:\Users\Josemiles\Desktop\GaitLab\src\Datasets\merged_data.csv")  

In [ ]:
df.shape

In [ ]:
df.isna().sum(axis=1)

In [ ]:
df.dropna(inplace=True)
df.shape

In [ ]:
df.head()

In [ ]:
def calculate_joint_angle(point_a, point_b, point_c):
    try:
        point_a, point_b, point_c = np.array(point_a), np.array(point_b), np.array(point_c)
        
        # Check for invalid points (NaN, inf, or None)
        if (np.any(np.isnan([point_a, point_b, point_c])) or 
            np.any(np.isinf([point_a, point_b, point_c]))):
            return 90.0  # Default neutral angle for timeseries continuity
        
        vector_ba = point_a - point_b
        vector_bc = point_c - point_b
        
        # Calculate norms with safety check
        norm_ba = np.linalg.norm(vector_ba)
        norm_bc = np.linalg.norm(vector_bc)
        
        if norm_ba < 1e-8 or norm_bc < 1e-8:
            return 90.0  # Default neutral angle
        
        # Calculate cosine with robust normalization
        cos_angle = np.dot(vector_ba, vector_bc) / (norm_ba * norm_bc)
        
        # Ensure cos_angle is within valid range for arccos
        cos_angle = np.clip(cos_angle, -1.0, 1.0)
        
        # Calculate angle and ensure it's finite
        angle_radians = np.arccos(cos_angle)
        angle_degrees = np.degrees(angle_radians)
        
        # Final safety check for timeseries integrity
        if np.isnan(angle_degrees) or np.isinf(angle_degrees):
            return 90.0
            
        return float(angle_degrees)
        
    except Exception:
        # Fallback for any unexpected errors - maintain timeseries continuity
        return 90.0


In [ ]:
#converting x_,y_,z_ landmark indexs to equivalent landmark names
landmark_names = {
    0: 'nose',
    1: 'left_eye_inner',
    2: 'left_eye',
    3: 'left_eye_outer',
    4: 'right_eye_inner',
    5: 'right_eye',
    6: 'right_eye_outer',
    7: 'left_ear',
    8: 'right_ear',
    9: 'mouth_left',
    10: 'mouth_right',
    11: 'left_shoulder',
    12: 'right_shoulder',
    13: 'left_elbow',
    14: 'right_elbow',
    15: 'left_wrist',
    16: 'right_wrist',
    23: 'left_hip',
    24: 'right_hip',
    25: 'left_knee',
    26: 'right_knee',
    27: 'left_ankle',
    28: 'right_ankle',
    29: 'left_heel',
    30: 'right_heel',
    31: 'left_foot_index',
    32: 'right_foot_index'
}

In [ ]:
df.columns

In [ ]:
df.columns = df.columns.map(
    lambda col: (
        f"{col.split('_')[0]}_{landmark_names[int(col.split('_')[1])]}"
        if len(col.split('_')) > 1 
        and col.split('_')[1].isdigit() 
        and int(col.split('_')[1]) in landmark_names
        else col
    )
)
df.columns

In [ ]:
#calculating angles for joints
df['left_knee_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_left_hip'], row['y_left_hip'], row['z_left_hip']),
        (row['x_left_knee'], row['y_left_knee'], row['z_left_knee']),
        (row['x_left_ankle'], row['y_left_ankle'], row['z_left_ankle'])
    ), axis=1
)
df['right_knee_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_right_hip'], row['y_right_hip'], row['z_right_hip']),
        (row['x_right_knee'], row['y_right_knee'], row['z_right_knee']),
        (row['x_right_ankle'], row['y_right_ankle'], row['z_right_ankle'])
    ), axis=1
)
df['left_elbow_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_left_shoulder'], row['y_left_shoulder'], row['z_left_shoulder']),
        (row['x_left_elbow'], row['y_left_elbow'], row['z_left_elbow']),
        (row['x_left_wrist'], row['y_left_wrist'], row['z_left_wrist'])
    ), axis=1
)
df['right_elbow_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_right_shoulder'], row['y_right_shoulder'], row['z_right_shoulder']),
        (row['x_right_elbow'], row['y_right_elbow'], row['z_right_elbow']),
        (row['x_right_wrist'], row['y_right_wrist'], row['z_right_wrist'])
    ), axis=1
)
#hip angles
df['left_hip_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_left_shoulder'], row['y_left_shoulder'], row['z_left_shoulder']),
        (row['x_left_hip'], row['y_left_hip'], row['z_left_hip']),
        (row['x_left_knee'], row['y_left_knee'], row['z_left_knee'])
    ), axis=1
)
df['right_hip_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_right_shoulder'], row['y_right_shoulder'], row['z_right_shoulder']),
        (row['x_right_hip'], row['y_right_hip'], row['z_right_hip']),
        (row['x_right_knee'], row['y_right_knee'], row['z_right_knee'])
    ), axis=1
)
#shalder angles
df['left_shoulder_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_left_elbow'], row['y_left_elbow'], row['z_left_elbow']),
        (row['x_left_shoulder'], row['y_left_shoulder'], row['z_left_shoulder']),
        (row['x_left_hip'], row['y_left_hip'], row['z_left_hip'])
    ), axis=1
)
df['right_shoulder_angle'] = df.apply(
    lambda row: calculate_joint_angle(
        (row['x_right_elbow'], row['y_right_elbow'], row['z_right_elbow']),
        (row['x_right_shoulder'], row['y_right_shoulder'], row['z_right_shoulder']),
        (row['x_right_hip'], row['y_right_hip'], row['z_right_hip'])
    ), axis=1
)
df.head()

In [ ]:
#distinct label values
df['label'].unique()

In [ ]:
#bar graph of label distribution
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='label', palette='tab10')
plt.title('Distribution of Labels')
plt.xlabel('Label')
plt.ylabel('Count')
plt.show()

In [ ]:
#plotting the variation of mean angles for left knee against their appropriate labels
plt.figure(figsize=(12, 6))
sns.barplot(data=df, x='label', y='left_knee_angle', palette='tab10')
plt.title('Mean Left Knee Angle by Label')
plt.xlabel('Label')
plt.ylabel('Mean Left Knee Angle')
plt.show()

In [34]:
binary_df=np.load(r"C:\Users\Josemiles\Desktop\GaitLab\src\Datasets\ts_data\y_seq_len64_stride32.npy")
binary_df.shape

(2204,)

In [35]:
binary_df

array(['KOA_Early', 'KOA_Early', 'KOA_Early', ..., 'NonAssistive',
       'NonAssistive', 'NonAssistive'], dtype='<U12')

In [37]:
binary_df=np.load(r"C:\Users\Josemiles\Desktop\GaitLab\src\Datasets\ts_data\x_seq_len64_stride32.npy")
binary_df.shape

(2204, 64, 21)